## Load the data

In [4]:
import os

embeddings_dir = "../data/embeddings/"
all_files = []
for file in os.listdir(embeddings_dir):
    if file.endswith(".json"):
        all_files.append(os.path.join(embeddings_dir, file))

print(f"Found {len(all_files)} embedding files.")

Found 5 embedding files.


In [5]:
import json

chunks = []
questions = []
chunk_id_counter = 0

for file_path in all_files:
    with open(file_path, "r") as f:
        data = json.load(f)

    file_chunks = data["chunks"]
    file_questions = data["question_answer_pairs"]

    old_to_new_id_map = {}

    # Update chunk IDs and add to the main list
    for chunk in file_chunks:
        old_id = chunk["id"]
        new_id = chunk_id_counter
        old_to_new_id_map[old_id] = new_id
        chunk["id"] = new_id
        chunks.append(chunk)
        chunk_id_counter += 1

    # Update chunk_id in questions and add to the main list
    for question in file_questions:
        old_chunk_id = question["chunk_id"]
        if old_chunk_id in old_to_new_id_map:
            question["chunk_id"] = old_to_new_id_map[old_chunk_id]
            questions.append(question)

print(f"Number of chunks    : {len(chunks)}")
print(f"Number of questions : {len(questions)}")

Number of chunks    : 120
Number of questions : 1209


## Evaluation metrics

There are several metrics commonly used to benchmark embedding models. In this notebook, you will use the following metrics:

- **Mean Reciprocal Rank (MRR)**
- **Recall@K**
- **Normalized Discounted Cumulative Gain (NDCG)**

For more details and additional metrics, see the [ranx metrics documentation](https://amenra.github.io/ranx/metrics/).

### Mean Reciprocal Rank (MRR)

MRR measures how quickly the first relevant result appears in the ranked list. The equation is:

$$
\text{MRR} = \frac{1}{\text{rank}}
$$

where **rank** is the position of the first relevant document.

In [6]:
import math


def calculate_mrr(rank: int | None) -> float:
    if rank is None:
        return 0.0
    return 1.0 / rank

### Recall@K

Recall@K indicates whether the relevant document appears within the top-$K$ results.  
For datasets with one relevant document ($R=1$):

$$
\text{Recall@K} = \frac{r}{R}
$$

where $r$ is 1 if the relevant document is in the top-$K$, otherwise 0.


In [7]:
def calculate_recall_at_k(rank: int | None, k: int) -> float:
    if rank is None:
        return 0.0
    return 1.0 if rank <= k else 0.0

### Normalized Discounted Cumulative Gain (NDCG@K)

N[DCG](https://en.wikipedia.org/wiki/Discounted_cumulative_gain)@K evaluates ranking quality, considering the position of the relevant document.  
For a single relevant document:

$$
\text{DCG@K} = 
\begin{cases}
\frac{1}{\log_2(\text{rank} + 1)}, & \text{if rank} \leq K \\
0, & \text{otherwise}
\end{cases}
$$

$$
\text{IDCG@K} = 1
$$

$$
\text{NDCG@K} = \frac{\text{DCG@K}}{\text{IDCG@K}} = \text{DCG@K}
$$

In [8]:
def calculate_ndcg_at_k(rank: int | None, k: int) -> float:
    if rank is None:
        return 0.0

    if rank <= k:
        return 1.0 / math.log2(rank + 1)
    else:
        return 0.0

## Prepare data for evaluation

Get the list of embedding models.

In [9]:
models_to_benchmark = list(chunks[0]["embeddings"].keys())
print(models_to_benchmark)

['text-embedding-3-small', 'text-embedding-3-large', 'all-minilm-l6-v2', 'gemini-embedding-001', 'qwen3-embedding-0.6b', 'qwen3-embedding-8b', 'qwen3-embedding-4b']


Create a mapping from each question to its correct chunk.

In [10]:
import numpy as np

ground_truth = {i: question["chunk_id"] for i, question in enumerate(questions)}
ground_truth

{0: 0,
 1: 0,
 2: 0,
 3: 0,
 4: 0,
 5: 0,
 6: 0,
 7: 0,
 8: 0,
 9: 0,
 10: 0,
 11: 0,
 12: 0,
 13: 1,
 14: 1,
 15: 1,
 16: 1,
 17: 1,
 18: 1,
 19: 2,
 20: 2,
 21: 2,
 22: 2,
 23: 2,
 24: 2,
 25: 3,
 26: 3,
 27: 3,
 28: 3,
 29: 3,
 30: 3,
 31: 3,
 32: 3,
 33: 4,
 34: 4,
 35: 4,
 36: 4,
 37: 4,
 38: 4,
 39: 4,
 40: 5,
 41: 5,
 42: 5,
 43: 5,
 44: 5,
 45: 5,
 46: 5,
 47: 5,
 48: 6,
 49: 6,
 50: 6,
 51: 6,
 52: 6,
 53: 6,
 54: 6,
 55: 6,
 56: 7,
 57: 7,
 58: 7,
 59: 7,
 60: 7,
 61: 7,
 62: 7,
 63: 7,
 64: 8,
 65: 8,
 66: 8,
 67: 8,
 68: 8,
 69: 8,
 70: 8,
 71: 8,
 72: 8,
 73: 8,
 74: 9,
 75: 9,
 76: 9,
 77: 9,
 78: 9,
 79: 9,
 80: 9,
 81: 9,
 82: 9,
 83: 9,
 84: 9,
 85: 9,
 86: 9,
 87: 9,
 88: 9,
 89: 9,
 90: 9,
 91: 9,
 92: 9,
 93: 9,
 94: 10,
 95: 10,
 96: 10,
 97: 10,
 98: 10,
 99: 10,
 100: 10,
 101: 10,
 102: 10,
 103: 10,
 104: 11,
 105: 11,
 106: 11,
 107: 11,
 108: 11,
 109: 11,
 110: 11,
 111: 11,
 112: 11,
 113: 11,
 114: 12,
 115: 12,
 116: 12,
 117: 12,
 118: 12,
 119: 12,
 120

Build a list of chunk IDs to make sure that the IDs are in sequential order.

In [11]:
chunk_ids_list = [chunk["id"] for chunk in chunks]
chunk_ids_list

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119]

Pre-load all chunk and question embeddings into NumPy arrays to make similarity calculations efficient.

In [12]:
chunk_embeddings = {}
question_embeddings = {}

for model in models_to_benchmark:
    chunk_embeddings[model] = np.array([chunk["embeddings"][model] for chunk in chunks])
    question_embeddings[model] = np.array(
        [question["embeddings"][model] for question in questions]
    )

In [13]:
chunk_embeddings["gemini-embedding-001"].shape

(120, 3072)

In [14]:
question_embeddings["gemini-embedding-001"].shape

(1209, 3072)

## Run the manual benchmark

This loop runs the benchmark for each embedding model:

- For each model, it calculates the cosine similarity between every question and all chunks.
- For each question, it ranks the chunks by similarity and finds the position (rank) of the correct chunk.
- It computes four metrics (MRR, Recall@1, Recall@5, NDCG@5) for each question using the rank.
- After processing all questions, it averages the scores for each metric and stores the results for the model.

This way, you get a summary of how well each model retrieves the correct chunk for all questions.

In [15]:
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity


def get_rank(ranked_chunk_ids: list[int], correct_chunk_id: int) -> int | None:
    try:
        return ranked_chunk_ids.index(correct_chunk_id) + 1
    except ValueError:
        return None


benchmark_results = {}
for model_name in tqdm(models_to_benchmark, total=len(models_to_benchmark)):
    all_question_embeddings = question_embeddings[model_name]
    all_chunk_embeddings = chunk_embeddings[model_name]
    similarity_matrix = cosine_similarity(all_question_embeddings, all_chunk_embeddings)

    all_mrr_scores = []
    all_recall_1_scores = []
    all_recall_5_scores = []
    all_ndcg_5_scores = []

    for i in range(len(questions)):
        correct_chunk_id = ground_truth[i]
        scores_for_this_question = similarity_matrix[i]

        chunk_scores = list(zip(chunk_ids_list, scores_for_this_question))
        sorted_chunk_scores = sorted(
            chunk_scores, key=lambda item: item[1], reverse=True
        )
        ranked_chunk_ids = [chunk_id for chunk_id, _ in sorted_chunk_scores]

        rank = get_rank(ranked_chunk_ids, correct_chunk_id)
        all_mrr_scores.append(calculate_mrr(rank))
        all_recall_1_scores.append(calculate_recall_at_k(rank, k=1))
        all_recall_5_scores.append(calculate_recall_at_k(rank, k=5))
        all_ndcg_5_scores.append(calculate_ndcg_at_k(rank, k=5))

    benchmark_results[model_name] = {
        "mrr": np.mean(all_mrr_scores),
        "recall@1": np.mean(all_recall_1_scores),
        "recall@5": np.mean(all_recall_5_scores),
        "ndcg@5": np.mean(all_ndcg_5_scores),
    }

100%|██████████| 7/7 [00:00<00:00, 14.43it/s]


## Display results

Find the best scores for each metric to highlight them in the results table.

In [16]:
metrics = ["mrr", "recall@1", "recall@5", "ndcg@5"]
max_scores = {metric: -float("inf") for metric in metrics}
min_scores = {metric: float("inf") for metric in metrics}

for model_name, scores in benchmark_results.items():
    for metric in metrics:
        if scores[metric] > max_scores[metric]:
            max_scores[metric] = scores[metric]
        if scores[metric] < min_scores[metric]:
            min_scores[metric] = scores[metric]

print(max_scores)
print(min_scores)

{'mrr': np.float64(0.7424935325893551), 'recall@1': np.float64(0.6360628618693135), 'recall@5': np.float64(0.8941273779983457), 'ndcg@5': np.float64(0.767530255256212)}
{'mrr': np.float64(0.5396133238351807), 'recall@1': np.float64(0.4267990074441687), 'recall@5': np.float64(0.6575682382133995), 'ndcg@5': np.float64(0.5500896918946327)}


Show the benchmark results in a formatted table, highlighting the best and worst scores for each metric.

In [17]:
from rich.box import HEAVY
from rich.table import Table
from rich.console import Console

console = Console()
table = Table(show_header=True, header_style="bold bright_magenta", box=HEAVY)
table.add_column("Model", style="yellow", width=25)
for metric in metrics:
    table.add_column(metric, justify="right")

for model_name, scores in benchmark_results.items():
    row_data = [model_name]
    for metric in metrics:
        score = scores[metric]
        score_str = f"{score:.4f}"

        if score == max_scores[metric]:
            score_str = f"[bold bright_green]{score_str}[/bold bright_green]"
        elif score == min_scores[metric]:
            score_str = f"[bold bright_red]{score_str}[/bold bright_red]"

        row_data.append(score_str)

    table.add_row(*row_data)

console.print(table)

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┓
┃ Model                     ┃    mrr ┃ recall@1 ┃ recall@5 ┃ ndcg@5 ┃
┣━━━━━━━━━━━━━━━━━━━━━━━━━━━╋━━━━━━━━╋━━━━━━━━━━╋━━━━━━━━━━╋━━━━━━━━┫
┃ text-embedding-3-small    ┃ 0.6972 ┃   0.5790 ┃   0.8544 ┃ 0.7261 ┃
┃ text-embedding-3-large    ┃ 0.6994 ┃   0.5773 ┃   0.8536 ┃ 0.7271 ┃
┃ all-minilm-l6-v2          ┃ 0.5396 ┃   0.4268 ┃   0.6576 ┃ 0.5501 ┃
┃ gemini-embedding-001      ┃ 0.7375 ┃   0.6303 ┃   0.8792 ┃ 0.7639 ┃
┃ qwen3-embedding-0.6b      ┃ 0.7276 ┃   0.6112 ┃   0.8801 ┃ 0.7560 ┃
┃ qwen3-embedding-8b        ┃ 0.7328 ┃   0.6129 ┃   0.8941 ┃ 0.7643 ┃
┃ qwen3-embedding-4b        ┃ 0.7425 ┃   0.6361 ┃   0.8784 ┃ 0.7675 ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━┻━━━━━━━━┻━━━━━━━━━━┻━━━━━━━━━━┻━━━━━━━━┛

If you want to sort the models by their average score across all metrics, you can calculate the average for each model and then sort them accordingly. This way, you can see which models perform best overall, rather than just on individual metrics.

In [18]:
from rich.box import HEAVY
from rich.table import Table
from rich.console import Console

console = Console()
table = Table(show_header=True, header_style="bold bright_magenta", box=HEAVY)
table.add_column("Model", style="yellow", width=25)
for metric in metrics:
    table.add_column(metric, justify="right")
table.add_column("Average", justify="right", style="bold cyan")

model_averages = {}
for model_name, scores in benchmark_results.items():
    average_score = np.mean([scores[metric] for metric in metrics])
    model_averages[model_name] = average_score

sorted_models = sorted(model_averages.items(), key=lambda x: x[1], reverse=True)
best_model_name = sorted_models[0][0]

for model_name, avg_score in sorted_models:
    model_name_cell_value = model_name
    if model_name == best_model_name:
        model_name_cell_value = f"[bold bright_green]{model_name}[/bold bright_green]"

    scores = benchmark_results[model_name]
    row_data = [model_name_cell_value]

    for metric in metrics:
        score = scores[metric]
        score_str = f"{score:.4f}"

        if score == max_scores[metric]:
            score_str = f"[bold bright_green]{score_str}[/bold bright_green]"
        elif score == min_scores[metric]:
            score_str = f"[bold bright_red]{score_str}[/bold bright_red]"

        row_data.append(score_str)

    row_data.append(f"{avg_score:.4f}")
    table.add_row(*row_data)

console.print(table)

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┓
┃ Model                     ┃    mrr ┃ recall@1 ┃ recall@5 ┃ ndcg@5 ┃ Average ┃
┣━━━━━━━━━━━━━━━━━━━━━━━━━━━╋━━━━━━━━╋━━━━━━━━━━╋━━━━━━━━━━╋━━━━━━━━╋━━━━━━━━━┫
┃ qwen3-embedding-4b        ┃ 0.7425 ┃   0.6361 ┃   0.8784 ┃ 0.7675 ┃  0.7561 ┃
┃ gemini-embedding-001      ┃ 0.7375 ┃   0.6303 ┃   0.8792 ┃ 0.7639 ┃  0.7527 ┃
┃ qwen3-embedding-8b        ┃ 0.7328 ┃   0.6129 ┃   0.8941 ┃ 0.7643 ┃  0.7510 ┃
┃ qwen3-embedding-0.6b      ┃ 0.7276 ┃   0.6112 ┃   0.8801 ┃ 0.7560 ┃  0.7437 ┃
┃ text-embedding-3-large    ┃ 0.6994 ┃   0.5773 ┃   0.8536 ┃ 0.7271 ┃  0.7144 ┃
┃ text-embedding-3-small    ┃ 0.6972 ┃   0.5790 ┃   0.8544 ┃ 0.7261 ┃  0.7142 ┃
┃ all-minilm-l6-v2          ┃ 0.5396 ┃   0.4268 ┃   0.6576 ┃ 0.5501 ┃  0.5435 ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━┻━━━━━━━━┻━━━━━━━━━━┻━━━━━━━━━━┻━━━━━━━━┻━━━━━━━━━┛